In [1]:
#KIST과제1-3
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
from itertools import product



c:\Users\L3352\.conda\envs\new2\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\L3352\.conda\envs\new2\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\L3352\.conda\envs\new2\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [4]:
class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, planes * self.expansion, kernel_size=1,
                               bias=False)
        self.bn3 = nn.BatchNorm2d(planes * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out




In [5]:
class ResNet(nn.Module):

    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes =planes * block.expansion  # 이 부분이 빠졌었네요
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
          x = self.conv1(x)
          x = self.bn1(x)
          x = self.relu(x)
          x = self.maxpool(x)

          x = self.layer1(x)
          x = self.layer2(x)
          x = self.layer3(x)
          x = self.layer4(x)

          x = self.avgpool(x)
          x = x.view(x.size(0), -1)
          x = self.fc(x)

          return x

      

In [6]:
def imshow(img):
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


In [7]:
# 데이터 전처리
transform = transforms.Compose([
    transforms.RandomResizedCrop(224),  # 이미지를 무작위로 자르고 224x224로 크기를 변경
    transforms.RandomHorizontalFlip(),  # 50% 확률로 이미지를 수평으로 뒤집음
    transforms.RandomRotation(10),      # -10에서 10도 사이의 무작위 회전
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# 데이터셋 및 데이터 로더 설정
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

trainloader = DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)
testloader = DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)



Files already downloaded and verified
Files already downloaded and verified


In [8]:
# ResNet 모델 인스턴스 생성
net = ResNet(Bottleneck, [3, 4, 6, 3])
net.to('cuda')

# 손실 함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()

# 모델 학습
num_epochs = 10
learning_rates = [0.01, 0.001, 0.0001]
batch_sizes = [32, 64, 128]

train_losses_all = []  # 모든 learning rate와 batch size에 대한 train loss를 저장
test_losses_all = []  # 모든 learning rate와 batch size에 대한 test loss를 저장
train_accuracies_all = []  # 모든 learning rate와 batch size에 대한 train accuracy를 저장
test_accuracies_all = []  # 모든 learning rate와 batch size에 대한 test accuracy를 저장


for lr in learning_rates:
    for bs in batch_sizes:
        optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)  # 스케줄러 추가
        trainloader = DataLoader(trainset, batch_size=bs, shuffle=True, num_workers=2)
        testloader = DataLoader(testset, batch_size=bs, shuffle=False, num_workers=2)

        train_losses = []   # Loss를 기록하기 위한 리스트
        test_losses = []
        train_accuracies = []  # 정확도를 기록하기 위한 리스트
        test_accuracies = []

        for epoch in range(num_epochs):
            running_loss = 0.0
            correct_train = 0
            total_train = 0
            net.train()

            
            
            for i, data in enumerate(trainloader, 0):
                inputs, labels = data
                inputs, labels = inputs.to('cuda'), labels.to('cuda')
                optimizer.zero_grad()

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)   # 정확도 계산을 위한 부분
                total_train += labels.size(0)
                correct_train += (predicted == labels).sum().item()
                train_accuracy = 100 * correct_train / total_train

                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

                running_loss += loss.item()

                del inputs, labels, outputs
            scheduler.step()  # 스케줄러 스텝 업데이트

            train_loss.append(running_loss / (i + 1))  # train_loss에 추가
            train_accuracies.append(train_accuracy)  # train_accuracy에 추가

            train_losses.append(np.mean(train_loss))  # 에포크의 끝에서 train_losses에 추가

            running_loss = 0.0
            correct_test = 0
            total_test = 0
            net.eval()

            test_loss = []  # 에포크의 시작에서 test_loss 리스트 초기화
            
            with torch.no_grad():
                for i, data in enumerate(testloader, 0):
                    inputs, labels = data
                    inputs, labels = inputs.to('cuda'), labels.to('cuda')

                    outputs = net(inputs)
                    _, predicted = torch.max(outputs.data, 1)   # 정확도 계산을 위한 부분
                    total_test += labels.size(0)
                    correct_test += (predicted == labels).sum().item()
                    test_accuracy = 100 * correct_test / total_test

                    loss = criterion(outputs, labels)

                    running_loss += loss.item()

                    del inputs, labels, outputs
                
                test_loss.append(running_loss / (i + 1))  # test_loss에 추가
                test_accuracies.append(test_accuracy)  # test_accuracy에 추가

            test_losses.append(np.mean(test_loss))  # 에포크의 끝에서 test_losses에 추가

        print(f"Learning Rate: {lr}, Batch Size: {bs}, Train Loss: {train_losses[-1]:.4f}, Test Loss: {test_losses[-1]:.4f}, Train Accuracy: {train_accuracies[-1]:.2f}%, Test Accuracy: {test_accuracies[-1]:.2f}%")
        # 학습이 끝난 후 각 결과를 저장
        train_losses_all.append(train_losses)
        test_losses_all.append(test_losses)
        train_accuracies_all.append(train_accuracies)
        test_accuracies_all.append(test_accuracies)




# Loss 그래프 그리기
for i, params in enumerate(param_combinations):
    lr, bs = params
    plt.figure(i)
    plt.plot(train_losses_all[i], label='Train Loss')
    plt.plot(test_losses_all[i], label='Test Loss')
    plt.title(f'Learning Rate: {lr}, Batch Size: {bs}')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

# 정확도 그래프 그리기
for i, params in enumerate(param_combinations):
    lr, bs = params
    plt.figure(i)
    plt.plot(train_accuracies_all[i], label='Train Accuracy')
    plt.plot(test_accuracies_all[i], label='Test Accuracy')
    plt.title(f'Learning Rate: {lr}, Batch Size: {bs}')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.show()


       




NameError: name 'train_loss' is not defined